In [24]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

def simular_datos_iot(n=1000, sensores=3, intervalo_segundos=60, semilla=1989):
    np.random.seed(semilla)
    registros = []
    inicio = datetime.now()

    for sensor_id in range(1, sensores + 1):
        tiempo = np.arange(n)

        # Bases por sensor
        base_temp = np.random.uniform(20, 30)
        base_hum = np.random.uniform(45, 75)
        base_co2 = np.random.uniform(370, 430)

        # Drift
        drift_temp = 0.005 * tiempo
        drift_hum = -0.003 * tiempo
        drift_co2 = 0.006 * tiempo

        # Ruido: gaussiano + senoidal + drift
        temperatura = (
            base_temp
            + np.random.normal(0, 7, size=n)
            + 10 * np.sin(2 * np.pi * tiempo / 1440)
            + drift_temp
        )

        humedad = (
            base_hum
            + np.random.normal(0, 10, size=n)
            + 10 * np.sin(2 * np.pi * tiempo / 2880 + sensor_id)
            + drift_hum
        )

        co2 = (
            base_co2
            + np.random.normal(0, 20, size=n)
            + 30 * np.sin(2 * np.pi * tiempo / 720 + sensor_id)
            + drift_co2
        )

        # Eventos extremos (~15%)
        indices_extremos = np.random.choice(n, size=int(n * 0.15), replace=False)
        temperatura[indices_extremos] += np.random.normal(10, 5, size=len(indices_extremos))
        humedad[indices_extremos] -= np.random.normal(25, 10, size=len(indices_extremos))
        co2[indices_extremos] += np.random.normal(250, 80, size=len(indices_extremos))

        # NaNs (~5%)
        for col in [temperatura, humedad, co2]:
            nan_indices = np.random.choice(n, size=int(n * 0.05), replace=False)
            col[nan_indices] = np.nan

        # Variable irrelevante (distractora)
        presion = np.random.normal(1013, 20, size=n)

        # Etiqueta zona crítica (más sensible)
        zona_critica = (
            ((temperatura > 28) & (humedad < 60)) |
            (co2 > 480)
        ).astype(int)

        # Agregar ruido en la etiqueta (~10% de inversión aleatoria)
        num_ruido = int(n * 0.10)
        indices_ruido = np.random.choice(n, size=num_ruido, replace=False)
        zona_critica[indices_ruido] = 1 - zona_critica[indices_ruido]  # Invierte 0↔1

        # Timestamp con jitter
        timestamps = [
            inicio + timedelta(seconds=i * intervalo_segundos + random.randint(-10, 10))
            for i in range(n)
        ]

        sensor_df = pd.DataFrame({
            'timestamp': timestamps,
            'sensor_id': sensor_id,
            'temperatura': temperatura,
            'humedad': humedad,
            'co2': co2,
            'presion': presion,
            'zona_critica': zona_critica
        })

        registros.append(sensor_df)

    return pd.concat(registros, ignore_index=True)

# Ejecutar y guardar
if __name__ == "__main__":
    df = simular_datos_iot(n=3000, sensores=4, intervalo_segundos=60)
    df.to_csv("Datos/sensores.csv", index=False)
    print("Datos Generados")

Datos Generados
